In [1]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
import sklearn.model_selection
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import pandas as pd

In [2]:
dataset_path = 'Admission_Predict_Ver1.1.csv'
raw_dataset = pd.read_csv(dataset_path, sep=",")

dataset = raw_dataset.copy()
dataset.info()
dataset.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
Serial No.           500 non-null int64
GRE Score            500 non-null int64
TOEFL Score          500 non-null int64
University Rating    500 non-null int64
SOP                  500 non-null float64
LOR                  500 non-null float64
CGPA                 500 non-null float64
Research             500 non-null int64
Chance of Admit      500 non-null float64
dtypes: float64(4), int64(5)
memory usage: 35.2 KB


,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


In [16]:
dataset = raw_dataset.copy() # Borrar el raw copy, es para poder hacer prints sobre esto y no tener datos distintos

# Split into vectors the dataset. y_all being the chance, x_all the other cols
# Y also gets categorized in 5 classes as output asks for.
bins = pd.cut(dataset.pop("Chance of Admit "), bins=np.arange(0, 1.1, 0.2), labels=[0,1,2,3,4])
y_all = keras.utils.to_categorical(bins, num_classes=5)
x_all = dataset

# Split dataset into train and tests. 33% used as asked.
x_train, x_valid, y_train, y_valid = sklearn.model_selection.train_test_split(x_all, y_all, test_size=0.33, random_state=0)

In [21]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation=tf.nn.relu, input_shape=tuple([x_train.shape[1]])),
    layers.Dense(64, activation=tf.nn.relu),
    layers.Dense(5)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mean_squared_error',
                optimizer=optimizer,
                metrics=['mean_absolute_error', 'mean_squared_error'])
  return model

In [22]:
model = build_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 64)                576       
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 325       
Total params: 5,061
Trainable params: 5,061
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.fit(x_train, y_train, batch_size=1024, epochs=200, validation_data=(x_valid, y_valid), verbose=1)

Train on 335 samples, validate on 165 samples
Epoch 1/200
335/335 [==============================] - 0s 443us/sample - loss: 3447.9363 - mean_absolute_error: 49.6554 - mean_squared_error: 3447.9363 - val_loss: 1475.8605 - val_mean_absolute_error: 31.9654 - val_mean_squared_error: 1475.8605
Epoch 2/200
335/335 [==============================] - 0s 15us/sample - loss: 1456.7012 - mean_absolute_error: 31.7279 - mean_squared_error: 1456.7012 - val_loss: 788.9691 - val_mean_absolute_error: 23.5954 - val_mean_squared_error: 788.9691
Epoch 3/200
335/335 [==============================] - 0s 22us/sample - loss: 769.4077 - mean_absolute_error: 23.2549 - mean_squared_error: 769.4077 - val_loss: 440.1341 - val_mean_absolute_error: 17.5895 - val_mean_squared_error: 440.1341
Epoch 4/200
335/335 [==============================] - 0s 18us/sample - loss: 421.0185 - mean_absolute_error: 17.1184 - mean_squared_error: 421.0185 - val_loss: 263.1714 - val_mean_absolute_error: 13.5903 - val_mean_squared_err

Epoch 36/200
335/335 [==============================] - 0s 16us/sample - loss: 15.8342 - mean_absolute_error: 3.1379 - mean_squared_error: 15.8342 - val_loss: 17.7332 - val_mean_absolute_error: 3.4665 - val_mean_squared_error: 17.7332
Epoch 37/200
335/335 [==============================] - 0s 15us/sample - loss: 16.5360 - mean_absolute_error: 3.3775 - mean_squared_error: 16.5360 - val_loss: 17.0946 - val_mean_absolute_error: 3.3112 - val_mean_squared_error: 17.0946
Epoch 38/200
335/335 [==============================] - 0s 28us/sample - loss: 15.4276 - mean_absolute_error: 3.1060 - mean_squared_error: 15.4276 - val_loss: 16.5980 - val_mean_absolute_error: 3.3428 - val_mean_squared_error: 16.5980
Epoch 39/200
335/335 [==============================] - 0s 11us/sample - loss: 15.4720 - mean_absolute_error: 3.2502 - mean_squared_error: 15.4720 - val_loss: 16.8208 - val_mean_absolute_error: 3.2901 - val_mean_squared_error: 16.8208
Epoch 40/200
335/335 [==============================] - 0s 1

Epoch 71/200
335/335 [==============================] - 0s 19us/sample - loss: 22.2560 - mean_absolute_error: 4.2322 - mean_squared_error: 22.2560 - val_loss: 19.3901 - val_mean_absolute_error: 3.9247 - val_mean_squared_error: 19.3901
Epoch 72/200
335/335 [==============================] - 0s 23us/sample - loss: 18.8838 - mean_absolute_error: 3.9199 - mean_squared_error: 18.8838 - val_loss: 15.7686 - val_mean_absolute_error: 3.5037 - val_mean_squared_error: 15.7686
Epoch 73/200
335/335 [==============================] - 0s 13us/sample - loss: 15.4777 - mean_absolute_error: 3.5322 - mean_squared_error: 15.4777 - val_loss: 13.4828 - val_mean_absolute_error: 3.2546 - val_mean_squared_error: 13.4828
Epoch 74/200
335/335 [==============================] - 0s 12us/sample - loss: 13.0333 - mean_absolute_error: 3.2287 - mean_squared_error: 13.0333 - val_loss: 12.4059 - val_mean_absolute_error: 2.9941 - val_mean_squared_error: 12.4059
Epoch 75/200
335/335 [==============================] - 0s 1

Epoch 106/200
335/335 [==============================] - 0s 28us/sample - loss: 12.6546 - mean_absolute_error: 2.9353 - mean_squared_error: 12.6546 - val_loss: 13.9504 - val_mean_absolute_error: 3.1311 - val_mean_squared_error: 13.9504
Epoch 107/200
335/335 [==============================] - 0s 14us/sample - loss: 13.9909 - mean_absolute_error: 3.1171 - mean_squared_error: 13.9909 - val_loss: 14.7858 - val_mean_absolute_error: 3.2601 - val_mean_squared_error: 14.7858
Epoch 108/200
335/335 [==============================] - 0s 13us/sample - loss: 14.5200 - mean_absolute_error: 3.1734 - mean_squared_error: 14.5200 - val_loss: 16.7272 - val_mean_absolute_error: 3.2659 - val_mean_squared_error: 16.7272
Epoch 109/200
335/335 [==============================] - 0s 14us/sample - loss: 16.6265 - mean_absolute_error: 3.2871 - mean_squared_error: 16.6265 - val_loss: 21.4083 - val_mean_absolute_error: 3.6046 - val_mean_squared_error: 21.4083
Epoch 110/200
335/335 [==============================] -

Epoch 141/200
335/335 [==============================] - 0s 11us/sample - loss: 13.4346 - mean_absolute_error: 3.1292 - mean_squared_error: 13.4346 - val_loss: 13.0183 - val_mean_absolute_error: 3.1490 - val_mean_squared_error: 13.0183
Epoch 142/200
335/335 [==============================] - 0s 17us/sample - loss: 12.8296 - mean_absolute_error: 3.1489 - mean_squared_error: 12.8296 - val_loss: 12.2642 - val_mean_absolute_error: 3.1295 - val_mean_squared_error: 12.2642
Epoch 143/200
335/335 [==============================] - 0s 17us/sample - loss: 12.1618 - mean_absolute_error: 3.1143 - mean_squared_error: 12.1618 - val_loss: 11.3435 - val_mean_absolute_error: 3.0010 - val_mean_squared_error: 11.3435
Epoch 144/200
335/335 [==============================] - 0s 16us/sample - loss: 11.1845 - mean_absolute_error: 2.9942 - mean_squared_error: 11.1845 - val_loss: 10.9721 - val_mean_absolute_error: 2.9105 - val_mean_squared_error: 10.9721
Epoch 145/200
335/335 [==============================] -

Epoch 176/200
335/335 [==============================] - 0s 23us/sample - loss: 18.8222 - mean_absolute_error: 4.0431 - mean_squared_error: 18.8222 - val_loss: 18.0288 - val_mean_absolute_error: 4.1005 - val_mean_squared_error: 18.0288
Epoch 177/200
335/335 [==============================] - 0s 11us/sample - loss: 18.2272 - mean_absolute_error: 4.1322 - mean_squared_error: 18.2272 - val_loss: 18.1510 - val_mean_absolute_error: 4.0854 - val_mean_squared_error: 18.1510
Epoch 178/200
335/335 [==============================] - 0s 14us/sample - loss: 17.8680 - mean_absolute_error: 4.0672 - mean_squared_error: 17.8680 - val_loss: 16.1051 - val_mean_absolute_error: 3.7245 - val_mean_squared_error: 16.1051
Epoch 179/200
335/335 [==============================] - 0s 18us/sample - loss: 16.1303 - mean_absolute_error: 3.7552 - mean_squared_error: 16.1303 - val_loss: 13.8594 - val_mean_absolute_error: 3.3632 - val_mean_squared_error: 13.8594
Epoch 180/200
335/335 [==============================] -

In [24]:
x_test = dataset
y_test = model.predict(x_test)

y_test

array([[-4.1196074 ,  4.398203  , -0.86772174,  1.6821026 ,  0.5752748 ],
       [-4.370328  ,  3.7977183 , -1.0003893 ,  1.9257702 ,  0.808509  ],
       [-4.234396  ,  3.3583138 , -1.0176489 ,  1.8351091 ,  0.86969316],
       ...,
       [-6.1830482 ,  5.018729  , -3.749781  ,  3.0011694 ,  3.123908  ],
       [-7.078846  ,  5.7159643 , -3.516783  ,  3.2880538 ,  2.6682072 ],
       [-6.5673885 ,  5.1500387 , -3.4776576 ,  3.2511904 ,  2.8487616 ]],
      dtype=float32)